In [1]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()



In [2]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
#Task 1
launch_site_list = []
launch_site_list = spacex_df['Launch Site'].unique().tolist()
launch_site_list.append('All Sites')
launch_site_list

['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40', 'All Sites']

In [4]:
#spacex_df[['Launch Site','class']].query('`class` ==1').groupby('Launch Site').sum()

In [10]:
# Create a dash application
app = dash.Dash(__name__)

In [11]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',options=[{'label': i, 'value': i} for i in launch_site_list]
                                            ,placeholder='Select a Launch Site here',value='All Sites',
                                            searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                
                                dcc.RangeSlider(id='payload-slider',min=0,max=10000,
                                step=1000,marks={0:'0',2500:'2500',5000:'5000',7500:'7500',10000:'10000'},
                                value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart',
             component_property='figure'),Input(component_id='site-dropdown',component_property='value')
             )
def draw_pie_chart(launch_site_location):
    if launch_site_location == 'All Sites':
        fig = px.pie(spacex_df,values = 'class',names='Launch Site')
    else:
        filtered_df = spacex_df.loc[spacex_df['Launch Site']==launch_site_location]
        #due to pie chart count only 1 but not 0, so we need to use value_counts
        fig = px.pie(filtered_df,values = filtered_df[['class']].value_counts(),names=pd.Series([0,1]))
    return fig
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
             [Input(component_id='site-dropdown',component_property='value'),
              Input(component_id='payload-slider',component_property='value')]
             )
def draw_scatter_chart(launch_site_location,payload_value):
    if launch_site_location == 'All Sites':
        filtered_df = spacex_df.loc[(spacex_df['Payload Mass (kg)']>payload_value[0])&(spacex_df['Payload Mass (kg)']<payload_value[1])]
        fig2 = px.scatter(filtered_df,x = 'Payload Mass (kg)',y='class', color="Booster Version Category")
    else:
        filtered_df = spacex_df.loc[(spacex_df['Launch Site']==launch_site_location)
                      &(spacex_df['Payload Mass (kg)']>payload_value[0])&(spacex_df['Payload Mass (kg)']<payload_value[1])]
        fig2 = px.scatter(filtered_df,x = 'Payload Mass (kg)',y='class', color="Booster Version Category")
    return fig2
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Aug/2021 15:45:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2021 15:45:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2021 15:45:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2021 15:45:51] "GET /_dash-component-suites/dash_core_components/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2021 15:45:51] "GET /_dash-component-suites/dash_core_components/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2021 15:45:51] "GET /_dash-component-suites/dash_core_components/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2021 15:45:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2021 15:45:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2021 15:45:51] "GET /_dash-component-suites/dash_core_components/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2021 15:45:55] "POST /_dash-update-component HTTP/1.1" 200 -
12